# This notebook segments Toronto into various neighborhoods

# PART 1

In [2]:
import pandas as pd 
import numpy as np

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
neighborhoods = pd.read_html(URL, header=0) #get info from wikipedia page
neighborhoods = neighborhoods[0] #remove unnecessary info

#remove rows where borough is not assigned
neighborhoods = neighborhoods.loc[(neighborhoods['Borough'] != 'Not assigned')] 

#confirm the postal codes are unique
print(neighborhoods.shape)
neighborhoods_count = neighborhoods.groupby('Postal Code')['Postal Code'].nunique()
print(neighborhoods_count.shape)

#ensure that neighborhoods are assigned as borough if not present
for id, row in neighborhoods.iterrows():
    if row.loc['Neighbourhood'] == 'Not assigned':
        neighborhoods.at[id, 'Neighbourhood'] = neighborhoods.at[id, 'Borough']
        print(str(row.loc['Borough']) + ' neighbourhood updated') 

#print the shape of the dataframe as requested in part 1
print('The shape of the neighborhoods dataframe is: ' + str(neighborhoods.shape))
print(neighborhoods.columns)

(103, 3)
(103,)
The shape of the neighborhoods dataframe is: (103, 3)
Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')


#  PART 2

In [3]:
pip install geocoder

     |████████████████████████████████| 98 kB 7.3 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
#geocoder wasnt working, imported csv using ibm functionality
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9d02acf03178425ebd1111cdf449e968 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='*****************************************************',
    ibm_auth_endpoint="*****************************************************",
    config=Config(signature_version='*****************************************************h'),
    endpoint_url='*****************************************************')

body = *****************************************************.get_object(Bucket='*****************************************************',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#join the tables at the intersection to get the latitude and longitude for the respective postal codes
loc_neighborhood = pd.merge(neighborhoods, df_data_2, on='Postal Code', how='inner')
#loc_neighborhood.drop(columns=['Latitude_x', 'Longitude_x'], inplace=True)
#loc_neighborhood.rename(columns={'Latitude_y': 'Latitude', 'Longitude_y': 'Longitude'}, inplace=True)
print(loc_neighborhood.head())

  Postal Code           Borough                                Neighbourhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  


# PART 3

In [14]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [8]:
pip install folium

     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [97]:
#load foursquare account info for get request -- remove credentials for privacy
CLIENT_ID = '*****************************************************'
CLIENT_SECRET = '*****************************************************' 
VERSION = '20180605'  

#convert neighborhoods to just toronto data frame and select postal code M5A - part of Downtown Toronto
toronto_neighborhoods = loc_neighborhood[loc_neighborhood['Borough'].str.contains('Toronto')]
toronto_neighborhoods.set_index('Postal Code', inplace=True)

#use getnearbyvenues function to get venues within the respective neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name) 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']  
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
toronot_places = getNearbyVenues(names=toronto_neighborhoods['Neighbourhood'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [98]:
toronot_places.head()
toronto_onehot = pd.get_dummies(toronot_places[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronot_places['Neighborhood']
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()
#view the common types of venues for a respective neighborhood

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.034483,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.000000,0.0,0.0,0.017241,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.017241,0.017241,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.000000,0.017241,0.051724,0.103448,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.000000,0.017241,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.017241,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017241,0.0,0.0,0.0,0.0,0.000000,0.034483,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.017241,0.017241,0.

In [99]:
#use most common venues to help understand main types of venues in the area
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Pet Store
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Garden,Burrito Place,Brewery,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place


In [104]:
#cluster the neighborhoods
k = 10
toronto_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_clustering)
#comment out adding in clust labels to avoid errors when re-running code, uncomment if needed
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#add the lat and long for mapping to the clusterd neighborhoods
toronot_places = toronot_places.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronot_places.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,1,Coffee Shop,Park,Bakery,Pub,Café
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,1,Coffee Shop,Park,Bakery,Pub,Café
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot,1,Coffee Shop,Park,Bakery,Pub,Café
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,1,Coffee Shop,Park,Bakery,Pub,Café
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,1,Coffee Shop,Park,Bakery,Pub,Café


In [114]:
toronot_places.groupby('Cluster Labels').count()



,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue
Cluster Labels,,,,,,,,,,,,
0,4,4,4,4,4,4,4,4,4,4,4,4
1,1154,1154,1154,1154,1154,1154,1154,1154,1154,1154,1154,1154
2,2,2,2,2,2,2,2,2,2,2,2,2
3,4,4,4,4,4,4,4,4,4,4,4,4
4,414,414,414,414,414,414,414,414,414,414,414,414
5,3,3,3,3,3,3,3,3,3,3,3,3
6,16,16,16,16,16,16,16,16,16,16,16,16
7,4,4,4,4,4,4,4,4,4,4,4,4
8,7,7,7,7,7,7,7,7,7,7,7,7


In [115]:
#most common clusters in toronto are labels 1,4 - this can bee seen by the counts above and the maps below
#view their features to undestand what is so common
toronot_places.loc[toronot_places['Cluster Labels'] == 1, toronot_places.columns[[1] + list(range(5, toronot_places.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue
0,43.654260,-79.362017,Bakery,1,Coffee Shop,Park,Bakery,Pub,Café
1,43.654260,-79.361809,Coffee Shop,1,Coffee Shop,Park,Bakery,Pub,Café
2,43.654260,-79.361149,Breakfast Spot,1,Coffee Shop,Park,Bakery,Pub,Café
3,43.654260,-79.358008,Distribution Center,1,Coffee Shop,Park,Bakery,Pub,Café
4,43.654260,-79.359874,Spa,1,Coffee Shop,Park,Bakery,Pub,Café
5,43.654260,-79.356980,Restaurant,1,Coffee Shop,Park,Bakery,Pub,Café
6,43.654260,-79.356211,Park,1,Coffee Shop,Park,Bakery,Pub,Café
7,43.654260,-79.359725,Gym / Fitness Center,1,Coffee Shop,Park,Bakery,Pub,Café
8,43.654260,-79.359323,Historic Site,1,Coffee Shop,Park,Bakery,Pub,Café
9,43.654260,-79.358127,Chocolate Shop,1,Coffee Shop,Park,Bakery,Pub,Café


In [116]:
toronot_places.loc[toronot_places['Cluster Labels'] == 4, toronot_places.columns[[1] + list(range(5, toronot_places.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Common Venue,2nd Common Venue,3rd Common Venue,4th Common Venue,5th Common Venue
496,43.669005,-79.439267,Bar,4,Pharmacy,Bakery,Bank,Pool,Bar
497,43.669005,-79.438728,Middle Eastern Restaurant,4,Pharmacy,Bakery,Bank,Pool,Bar
498,43.669005,-79.440754,Grocery Store,4,Pharmacy,Bakery,Bank,Pool,Bar
499,43.669005,-79.436533,Brewery,4,Pharmacy,Bakery,Bank,Pool,Bar
500,43.669005,-79.441791,Bakery,4,Pharmacy,Bakery,Bank,Pool,Bar
501,43.669005,-79.440265,Café,4,Pharmacy,Bakery,Bank,Pool,Bar
502,43.669005,-79.443603,Music Venue,4,Pharmacy,Bakery,Bank,Pool,Bar
503,43.669005,-79.437582,Bakery,4,Pharmacy,Bakery,Bank,Pool,Bar
504,43.669005,-79.446691,Supermarket,4,Pharmacy,Bakery,Bank,Pool,Bar
505,43.669005,-79.441698,Bank,4,Pharmacy,Bakery,Bank,Pool,Bar


In [118]:
#map the clusters using folium
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronot_places['Neighborhood Latitude'], toronot_places['Neighborhood Longitude'],toronot_places['Neighborhood'], toronot_places['Cluster Labels']):
    label = folium.Popup(str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#insert screenshot of map - sometimes doesnt load 
